In [1]:
# ! pip install catboost

In [2]:
# Stacking Startetr based on on Allstate Faron's Script
# Preprocessing from ogrellier

import pandas as pd
import numpy as np 
from scipy.stats import skew
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc 

In [3]:
NFOLDS = 3
SEED = 0
NROWS = None

In [4]:
data = pd.read_csv('~/다운로드/application_train.csv')
test = pd.read_csv('~/다운로드/application_test.csv')
prev = pd.read_csv('~/다운로드/previous_application.csv')

In [5]:
categorical_feats = [f for f in data.columns if data[f].dtype =='object']

for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])

gc.enable()

In [6]:
y_train = data['TARGET']
del data['TARGET']

In [7]:
prev_cat_feature = [f_ for f_ in prev.columns if  prev[f_].dtype=='object']

for f_ in prev_cat_feature:
    prev[f_], _ = pd.factorize(prev[f_])

In [8]:
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

In [9]:
x_train = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
x_test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [10]:
ntrain = x_train.shape[0]
ntest = x_test.shape[0]

In [11]:
excluded_feats = ['SK_ID_CURR']
features = [f_ for f_ in x_train.columns if f_ not in excluded_feats]

x_train = x_train[features]
x_test = x_test[features]

In [12]:
kf =KFold(n_splits= NFOLDS, shuffle=True, random_state=SEED)

In [13]:
class SKlearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)
    
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]

In [14]:
class CatBoostWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]

In [15]:
class LightGBMWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['feature_fraction_seed'] = seed
        params['bagging_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]

In [16]:
class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

In [17]:
def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf =np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)

In [18]:
et_params ={
    'n_jobs' : 16,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'verbosity': 1,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'nrounds': 200
}

catboost_params = {
    'iterations': 200,
    'learning_rate': 0.5,
    'depth': 3,
    'l2_leaf_reg': 40,
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.7,
    'scale_pos_weight': 5,
    'eval_metric': 'AUC',
    'od_type': 'Iter',
    'allow_writing_files': False
}

lightgbm_params = {
    'n_estimators': 200,
    'learning_rate': 0.1,
    'num_leaves': 123,
    'colsample_bytree': 0.8,
    'subsample': 0.9,
    'max_depth': 15,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'min_split_gain': 0.01,
    'min_child_weight': 2
}

In [19]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SKlearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SKlearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatBoostWrapper(clf=CatBoostClassifier, seed=SEED, params=catboost_params)
lg = LightGBMWrapper(clf=LGBMClassifier, seed= SEED, params=lightgbm_params)

In [20]:
xg_oof_train, xg_oof_test = get_oof(xg)

In [21]:
et_oof_train, et_oof_test = get_oof(et)

In [22]:
rf_oof_train, rf_oof_test = get_oof(rf)

In [23]:
cb_oof_train, cb_oof_test = get_oof(cb)

0:	total: 113ms	remaining: 22.4s
1:	total: 185ms	remaining: 18.3s
2:	total: 246ms	remaining: 16.1s
3:	total: 307ms	remaining: 15.1s
4:	total: 378ms	remaining: 14.7s
5:	total: 443ms	remaining: 14.3s
6:	total: 509ms	remaining: 14s
7:	total: 627ms	remaining: 15.1s
8:	total: 696ms	remaining: 14.8s
9:	total: 763ms	remaining: 14.5s
10:	total: 823ms	remaining: 14.1s
11:	total: 896ms	remaining: 14s
12:	total: 958ms	remaining: 13.8s
13:	total: 1.01s	remaining: 13.4s
14:	total: 1.07s	remaining: 13.2s
15:	total: 1.13s	remaining: 13s
16:	total: 1.19s	remaining: 12.8s
17:	total: 1.24s	remaining: 12.5s
18:	total: 1.28s	remaining: 12.2s
19:	total: 1.32s	remaining: 11.9s
20:	total: 1.38s	remaining: 11.7s
21:	total: 1.44s	remaining: 11.6s
22:	total: 1.49s	remaining: 11.4s
23:	total: 1.54s	remaining: 11.3s
24:	total: 1.59s	remaining: 11.1s
25:	total: 1.66s	remaining: 11.1s
26:	total: 1.76s	remaining: 11.3s
27:	total: 1.88s	remaining: 11.5s
28:	total: 1.93s	remaining: 11.4s
29:	total: 1.97s	remaining: 11

In [24]:
print(f'XG_CV: {sqrt(mean_squared_error(y_train, xg_oof_train))}')
print(f'ET-CV: {sqrt(mean_squared_error(y_train, et_oof_train))}')
print(f'RF-CV: {sqrt(mean_squared_error(y_train, rf_oof_train))}')
print(f'CB-CV: {sqrt(mean_squared_error(y_train, cb_oof_train))}')

XG_CV: 0.25966781672358985
ET-CV: 0.26296502347137407
RF-CV: 0.2629856213006646
CB-CV: 0.33101276561439696


In [25]:
x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test), axis=1)

In [26]:
print(f'{x_train.shape}, {x_test.shape}')

(307511, 4), (48744, 4)


In [27]:
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train, y_train)

LogisticRegression()

In [28]:
test['TARGET'] = logistic_regression.predict_proba(x_test)[:,1]
test[['SK_ID_CURR', 'TARGET']].to_csv('first_submission.csv', index=False, float_format='%.8f')